# Physics-Informed Neural Networks (PINNs) Tutorial

This notebook provides a comprehensive introduction to Physics-Informed Neural Networks (PINNs) for solving partial differential equations (PDEs). We'll implement and train PINNs to solve various types of PDEs.

## Table of Contents
1. [Introduction to PINNs](#introduction)
2. [Mathematical Background](#mathematical-background)
3. [Implementation](#implementation)
4. [Example 1: 1D Poisson Equation](#example-1)
5. [Example 2: 2D Poisson Equation](#example-2)
6. [Example 3: Burgers' Equation](#example-3)
7. [Advanced Topics](#advanced-topics)
8. [Conclusion](#conclusion)

## 1. Introduction to PINNs {#introduction}

Physics-Informed Neural Networks (PINNs) are a revolutionary approach to solving partial differential equations (PDEs) using deep learning. Unlike traditional numerical methods, PINNs:

- **Incorporate physics directly**: PDEs are embedded as regularization terms in the loss function
- **Handle complex geometries**: No need for mesh generation
- **Work with limited data**: Can solve PDEs with minimal or no training data
- **Provide smooth solutions**: Neural networks naturally produce smooth, differentiable solutions

### Key Advantages:
- Mesh-free approach
- Automatic differentiation for computing derivatives
- Integration of multiple physics constraints
- Uncertainty quantification capabilities

## 2. Mathematical Background {#mathematical-background}

### General PDE Form
Consider a general PDE of the form:
$$F(\mathbf{x}, u, \nabla u, \nabla^2 u, \ldots) = 0$$

where:
- $\mathbf{x}$ represents the input coordinates (spatial and/or temporal)
- $u$ is the solution we want to find
- $\nabla u$, $\nabla^2 u$ are the derivatives of $u$

### PINN Approach
1. **Neural Network Approximation**: Use a neural network $u_{\theta}(\mathbf{x})$ to approximate the solution
2. **Physics Loss**: Define the physics loss as:
   $$\mathcal{L}_{physics} = \frac{1}{N_{collocation}} \sum_{i=1}^{N_{collocation}} |F(\mathbf{x}_i, u_{\theta}(\mathbf{x}_i), \nabla u_{\theta}(\mathbf{x}_i), \ldots)|^2$$

3. **Boundary/Initial Conditions**: Add losses for boundary and initial conditions:
   $$\mathcal{L}_{BC} = \frac{1}{N_{BC}} \sum_{i=1}^{N_{BC}} |u_{\theta}(\mathbf{x}_{BC,i}) - u_{BC,i}|^2$$

4. **Total Loss**: Combine all losses:
   $$\mathcal{L}_{total} = \lambda_{physics} \mathcal{L}_{physics} + \lambda_{BC} \mathcal{L}_{BC} + \lambda_{data} \mathcal{L}_{data}$$

## 3. Implementation {#implementation}

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


### Basic PINN Architecture

Let's implement a simple PINN architecture:

In [ ]:
class SimplePINN(nn.Module):
    """
    A simple Physics-Informed Neural Network.
    """
    def __init__(self, input_dim=1, hidden_dim=50, output_dim=1, num_layers=4):
        super(SimplePINN, self).__init__()
        
        # Build the network
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.Tanh())
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Tanh())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
        # Initialize weights
        self.init_weights()
    
    def init_weights(self):
        """Initialize network weights using Xavier initialization."""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.network(x)

# Test the network
model = SimplePINN(input_dim=1, hidden_dim=32, output_dim=1, num_layers=3)
test_input = torch.randn(10, 1)
test_output = model(test_input)
print(f"Model input shape: {test_input.shape}")
print(f"Model output shape: {test_output.shape}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

### Automatic Differentiation Helper Function

PINNs rely heavily on automatic differentiation to compute derivatives of the neural network output with respect to its inputs:

In [ ]:
def compute_derivatives(u, x, order=1):
    """
    Compute derivatives of u with respect to x using automatic differentiation.
    
    Args:
        u: Neural network output
        x: Input coordinates (requires_grad=True)
        order: Order of derivative (1 or 2)
    
    Returns:
        List of derivatives with respect to each coordinate
    """
    grad_outputs = torch.ones_like(u)
    derivatives = []
    
    for i in range(x.shape[1]):
        if order == 1:
            # First derivative
            grad_u = torch.autograd.grad(u, x, grad_outputs=grad_outputs,
                                       create_graph=True, retain_graph=True)[0]
            derivative = grad_u[:, i:i+1]
        elif order == 2:
            # First derivative
            grad_u = torch.autograd.grad(u, x, grad_outputs=grad_outputs,
                                       create_graph=True, retain_graph=True)[0]
            u_x = grad_u[:, i:i+1]
            # Second derivative
            derivative = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x),
                                           create_graph=True, retain_graph=True)[0][:, i:i+1]
        else:
            raise NotImplementedError(f"Order {order} derivatives not implemented")
        
        derivatives.append(derivative)
    
    return derivatives

# Test automatic differentiation
x = torch.linspace(-1, 1, 100).reshape(-1, 1)
x.requires_grad_(True)

# Test function: u = sin(πx)
u_test = torch.sin(torch.pi * x)

# Compute derivatives
u_x = compute_derivatives(u_test, x, order=1)[0]
u_xx = compute_derivatives(u_test, x, order=2)[0]

# Analytical derivatives for comparison
u_x_analytical = torch.pi * torch.cos(torch.pi * x)
u_xx_analytical = -torch.pi**2 * torch.sin(torch.pi * x)

print(f"First derivative error: {torch.mean((u_x - u_x_analytical)**2).item():.2e}")
print(f"Second derivative error: {torch.mean((u_xx - u_xx_analytical)**2).item():.2e}")

## 4. Example 1: 1D Poisson Equation {#example-1}

Let's solve our first PDE: the 1D Poisson equation.

### Problem Setup
$$\frac{d^2 u}{dx^2} = -\pi^2 \sin(\pi x), \quad x \in [-1, 1]$$
$$u(-1) = u(1) = 0 \quad \text{(boundary conditions)}$$

**Analytical Solution**: $u(x) = \sin(\pi x)$

In [ ]:
class Poisson1D_PINN:
    """
    PINN for solving 1D Poisson equation.
    """
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.history = {'loss': [], 'pde_loss': [], 'bc_loss': []}
    
    def pde_loss(self, x_collocation):
        """Compute PDE residual loss."""
        x_collocation.requires_grad_(True)
        u = self.model(x_collocation)
        
        # Compute second derivative
        u_xx = compute_derivatives(u, x_collocation, order=2)[0]
        
        # Source term: f(x) = -π²sin(πx)
        f = -torch.pi**2 * torch.sin(torch.pi * x_collocation)
        
        # PDE residual: d²u/dx² - f = 0
        residual = u_xx - f
        return torch.mean(residual**2)
    
    def boundary_loss(self, x_boundary, u_boundary):
        """Compute boundary condition loss."""
        u_pred = self.model(x_boundary)
        return torch.mean((u_pred - u_boundary)**2)
    
    def train(self, x_collocation, x_boundary, u_boundary, epochs=5000, 
              lambda_pde=1.0, lambda_bc=100.0, print_every=1000):
        """Train the PINN."""
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            
            # Compute losses
            pde_loss = self.pde_loss(x_collocation)
            bc_loss = self.boundary_loss(x_boundary, u_boundary)
            
            # Total loss
            total_loss = lambda_pde * pde_loss + lambda_bc * bc_loss
            
            # Backward pass
            total_loss.backward()
            self.optimizer.step()
            
            # Record history
            self.history['loss'].append(total_loss.item())
            self.history['pde_loss'].append(pde_loss.item())
            self.history['bc_loss'].append(bc_loss.item())
            
            if (epoch + 1) % print_every == 0:
                print(f"Epoch {epoch+1:5d}: Total = {total_loss.item():.6e}, "
                      f"PDE = {pde_loss.item():.6e}, BC = {bc_loss.item():.6e}")

# Create training data
n_collocation = 1000
x_collocation = torch.rand(n_collocation, 1) * 2 - 1  # Random points in [-1, 1]

# Boundary conditions
x_boundary = torch.tensor([[-1.0], [1.0]], dtype=torch.float32)
u_boundary = torch.zeros_like(x_boundary)

# Create model and trainer
model_1d = SimplePINN(input_dim=1, hidden_dim=32, output_dim=1, num_layers=4)
trainer_1d = Poisson1D_PINN(model_1d)

print("Training 1D Poisson PINN...")
trainer_1d.train(x_collocation, x_boundary, u_boundary, epochs=3000)

In [ ]:
# Evaluate and visualize results
x_test = torch.linspace(-1, 1, 200).reshape(-1, 1)
with torch.no_grad():
    u_pred = model_1d(x_test)
    u_exact = torch.sin(torch.pi * x_test)

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Solution comparison
axes[0].plot(x_test.numpy(), u_pred.numpy(), 'b-', linewidth=2, label='PINN')
axes[0].plot(x_test.numpy(), u_exact.numpy(), 'r--', linewidth=2, label='Exact')
axes[0].scatter(x_boundary.numpy(), u_boundary.numpy(), c='red', s=50, zorder=5, label='BC')
axes[0].set_xlabel('x')
axes[0].set_ylabel('u(x)')
axes[0].set_title('1D Poisson Solution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error plot
error = torch.abs(u_pred - u_exact)
axes[1].plot(x_test.numpy(), error.numpy(), 'g-', linewidth=2)
axes[1].set_xlabel('x')
axes[1].set_ylabel('|Error|')
axes[1].set_title(f'Absolute Error (Max: {error.max().item():.2e})')
axes[1].grid(True, alpha=0.3)

# Training history
axes[2].semilogy(trainer_1d.history['loss'], label='Total Loss')
axes[2].semilogy(trainer_1d.history['pde_loss'], label='PDE Loss')
axes[2].semilogy(trainer_1d.history['bc_loss'], label='BC Loss')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss')
axes[2].set_title('Training History')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute errors
mse = torch.mean((u_pred - u_exact)**2).item()
rel_l2 = torch.norm(u_pred - u_exact) / torch.norm(u_exact)
print(f"Mean Squared Error: {mse:.2e}")
print(f"Relative L2 Error: {rel_l2.item():.2e}")

## 5. Example 2: 2D Poisson Equation {#example-2}

Now let's tackle a 2D problem:

### Problem Setup
$$\nabla^2 u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -2\pi^2 \sin(\pi x) \sin(\pi y)$$
$$u(x,y) = 0 \text{ on boundary of } [-1,1] \times [-1,1]$$

**Analytical Solution**: $u(x,y) = \sin(\pi x) \sin(\pi y)$

In [ ]:
class Poisson2D_PINN:
    """
    PINN for solving 2D Poisson equation.
    """
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.history = {'loss': [], 'pde_loss': [], 'bc_loss': []}
    
    def pde_loss(self, xy_collocation):
        """Compute PDE residual loss."""
        xy_collocation.requires_grad_(True)
        u = self.model(xy_collocation)
        
        # Compute second derivatives
        u_xx = compute_derivatives(u, xy_collocation, order=2)[0]  # ∂²u/∂x²
        u_yy = compute_derivatives(u, xy_collocation, order=2)[1]  # ∂²u/∂y²
        
        # Laplacian
        laplacian = u_xx + u_yy
        
        # Source term
        x, y = xy_collocation[:, 0:1], xy_collocation[:, 1:2]
        f = -2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
        
        # PDE residual
        residual = laplacian - f
        return torch.mean(residual**2)
    
    def boundary_loss(self, xy_boundary):
        """Compute boundary condition loss (homogeneous Dirichlet)."""
        u_pred = self.model(xy_boundary)
        return torch.mean(u_pred**2)
    
    def train(self, xy_collocation, xy_boundary, epochs=5000, 
              lambda_pde=1.0, lambda_bc=100.0, print_every=1000):
        """Train the PINN."""
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            
            # Compute losses
            pde_loss = self.pde_loss(xy_collocation)
            bc_loss = self.boundary_loss(xy_boundary)
            
            # Total loss
            total_loss = lambda_pde * pde_loss + lambda_bc * bc_loss
            
            # Backward pass
            total_loss.backward()
            self.optimizer.step()
            
            # Record history
            self.history['loss'].append(total_loss.item())
            self.history['pde_loss'].append(pde_loss.item())
            self.history['bc_loss'].append(bc_loss.item())
            
            if (epoch + 1) % print_every == 0:
                print(f"Epoch {epoch+1:5d}: Total = {total_loss.item():.6e}, "
                      f"PDE = {pde_loss.item():.6e}, BC = {bc_loss.item():.6e}")

# Create 2D training data
n_collocation = 2000
xy_collocation = torch.rand(n_collocation, 2) * 2 - 1  # Random points in [-1,1]²

# Boundary points
n_boundary = 400
# Left and right boundaries
xy_left = torch.cat([torch.full((n_boundary//4, 1), -1), 
                     torch.rand(n_boundary//4, 1) * 2 - 1], dim=1)
xy_right = torch.cat([torch.full((n_boundary//4, 1), 1), 
                      torch.rand(n_boundary//4, 1) * 2 - 1], dim=1)
# Bottom and top boundaries
xy_bottom = torch.cat([torch.rand(n_boundary//4, 1) * 2 - 1, 
                       torch.full((n_boundary//4, 1), -1)], dim=1)
xy_top = torch.cat([torch.rand(n_boundary//4, 1) * 2 - 1, 
                    torch.full((n_boundary//4, 1), 1)], dim=1)

xy_boundary = torch.cat([xy_left, xy_right, xy_bottom, xy_top], dim=0)

# Create model and trainer
model_2d = SimplePINN(input_dim=2, hidden_dim=64, output_dim=1, num_layers=5)
trainer_2d = Poisson2D_PINN(model_2d)

print("Training 2D Poisson PINN...")
trainer_2d.train(xy_collocation, xy_boundary, epochs=3000)

In [ ]:
# Evaluate and visualize 2D results
n_test = 50
x_test = torch.linspace(-1, 1, n_test)
y_test = torch.linspace(-1, 1, n_test)
X_test, Y_test = torch.meshgrid(x_test, y_test, indexing='ij')
xy_test = torch.stack([X_test.flatten(), Y_test.flatten()], dim=1)

with torch.no_grad():
    u_pred = model_2d(xy_test).reshape(n_test, n_test)
    u_exact = torch.sin(torch.pi * X_test) * torch.sin(torch.pi * Y_test)

# Convert to numpy for plotting
X_np = X_test.numpy()
Y_np = Y_test.numpy()
u_pred_np = u_pred.numpy()
u_exact_np = u_exact.numpy()
error_np = np.abs(u_pred_np - u_exact_np)

# Create 2D plots
fig = plt.figure(figsize=(16, 10))

# Predicted solution
ax1 = fig.add_subplot(2, 3, 1, projection='3d')
surf1 = ax1.plot_surface(X_np, Y_np, u_pred_np, cmap='viridis', alpha=0.8)
ax1.set_title('PINN Solution')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('u')

# Exact solution
ax2 = fig.add_subplot(2, 3, 2, projection='3d')
surf2 = ax2.plot_surface(X_np, Y_np, u_exact_np, cmap='viridis', alpha=0.8)
ax2.set_title('Exact Solution')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('u')

# Error
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
surf3 = ax3.plot_surface(X_np, Y_np, error_np, cmap='Reds', alpha=0.8)
ax3.set_title('Absolute Error')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('|Error|')

# Contour plots
ax4 = fig.add_subplot(2, 3, 4)
contour1 = ax4.contourf(X_np, Y_np, u_pred_np, levels=20, cmap='viridis')
ax4.set_title('PINN Solution (Contour)')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
plt.colorbar(contour1, ax=ax4)

ax5 = fig.add_subplot(2, 3, 5)
contour2 = ax5.contourf(X_np, Y_np, u_exact_np, levels=20, cmap='viridis')
ax5.set_title('Exact Solution (Contour)')
ax5.set_xlabel('x')
ax5.set_ylabel('y')
plt.colorbar(contour2, ax=ax5)

# Training history
ax6 = fig.add_subplot(2, 3, 6)
ax6.semilogy(trainer_2d.history['loss'], label='Total Loss')
ax6.semilogy(trainer_2d.history['pde_loss'], label='PDE Loss')
ax6.semilogy(trainer_2d.history['bc_loss'], label='BC Loss')
ax6.set_xlabel('Epoch')
ax6.set_ylabel('Loss')
ax6.set_title('Training History')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute errors
mse_2d = torch.mean((u_pred - u_exact)**2).item()
rel_l2_2d = torch.norm(u_pred - u_exact) / torch.norm(u_exact)
print(f"2D Poisson - Mean Squared Error: {mse_2d:.2e}")
print(f"2D Poisson - Relative L2 Error: {rel_l2_2d.item():.2e}")

## 6. Example 3: Burgers' Equation {#example-3}

Now let's solve a time-dependent nonlinear PDE - Burgers' equation:

### Problem Setup
$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$

with:
- Domain: $x \in [-1, 1]$, $t \in [0, 1]$
- Initial condition: $u(x, 0) = -\sin(\pi x)$
- Boundary conditions: $u(-1, t) = u(1, t) = 0$
- Viscosity: $\nu = 0.01$

In [ ]:
class Burgers_PINN:
    """
    PINN for solving Burgers' equation.
    """
    def __init__(self, model, nu=0.01):
        self.model = model
        self.nu = nu
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.history = {'loss': [], 'pde_loss': [], 'ic_loss': [], 'bc_loss': []}
    
    def pde_loss(self, xt_collocation):
        """Compute PDE residual loss."""
        xt_collocation.requires_grad_(True)
        u = self.model(xt_collocation)
        
        # Compute derivatives
        u_x = compute_derivatives(u, xt_collocation, order=1)[0]  # ∂u/∂x
        u_t = compute_derivatives(u, xt_collocation, order=1)[1]  # ∂u/∂t
        u_xx = compute_derivatives(u, xt_collocation, order=2)[0]  # ∂²u/∂x²
        
        # Burgers' equation residual: ∂u/∂t + u∂u/∂x - ν∂²u/∂x² = 0
        residual = u_t + u * u_x - self.nu * u_xx
        return torch.mean(residual**2)
    
    def initial_loss(self, x_initial, u_initial):
        """Compute initial condition loss."""
        u_pred = self.model(x_initial)
        return torch.mean((u_pred - u_initial)**2)
    
    def boundary_loss(self, xt_boundary):
        """Compute boundary condition loss."""
        u_pred = self.model(xt_boundary)
        return torch.mean(u_pred**2)
    
    def train(self, xt_collocation, x_initial, u_initial, xt_boundary, 
              epochs=5000, lambda_pde=1.0, lambda_ic=10.0, lambda_bc=10.0, 
              print_every=1000):
        """Train the PINN."""
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            
            # Compute losses
            pde_loss = self.pde_loss(xt_collocation)
            ic_loss = self.initial_loss(x_initial, u_initial)
            bc_loss = self.boundary_loss(xt_boundary)
            
            # Total loss
            total_loss = lambda_pde * pde_loss + lambda_ic * ic_loss + lambda_bc * bc_loss
            
            # Backward pass
            total_loss.backward()
            self.optimizer.step()
            
            # Record history
            self.history['loss'].append(total_loss.item())
            self.history['pde_loss'].append(pde_loss.item())
            self.history['ic_loss'].append(ic_loss.item())
            self.history['bc_loss'].append(bc_loss.item())
            
            if (epoch + 1) % print_every == 0:
                print(f"Epoch {epoch+1:5d}: Total = {total_loss.item():.6e}, "
                      f"PDE = {pde_loss.item():.6e}, IC = {ic_loss.item():.6e}, "
                      f"BC = {bc_loss.item():.6e}")

# Create Burgers' equation training data
n_collocation = 2000
x_col = torch.rand(n_collocation, 1) * 2 - 1  # x ∈ [-1, 1]
t_col = torch.rand(n_collocation, 1)          # t ∈ [0, 1]
xt_collocation = torch.cat([x_col, t_col], dim=1)

# Initial condition: u(x, 0) = -sin(πx)
n_initial = 100
x_initial = torch.linspace(-1, 1, n_initial).reshape(-1, 1)
t_initial = torch.zeros_like(x_initial)
xt_initial = torch.cat([x_initial, t_initial], dim=1)
u_initial = -torch.sin(torch.pi * x_initial)

# Boundary conditions: u(-1, t) = u(1, t) = 0
n_boundary = 100
t_boundary = torch.linspace(0, 1, n_boundary).reshape(-1, 1)
x_left = torch.full_like(t_boundary, -1)
x_right = torch.full_like(t_boundary, 1)
xt_boundary = torch.cat([
    torch.cat([x_left, t_boundary], dim=1),
    torch.cat([x_right, t_boundary], dim=1)
], dim=0)

# Create model and trainer
model_burgers = SimplePINN(input_dim=2, hidden_dim=64, output_dim=1, num_layers=6)
trainer_burgers = Burgers_PINN(model_burgers, nu=0.01)

print("Training Burgers' equation PINN...")
trainer_burgers.train(xt_collocation, xt_initial, u_initial, xt_boundary, epochs=3000)

In [ ]:
# Evaluate and visualize Burgers' equation results
n_x, n_t = 100, 50
x_test = torch.linspace(-1, 1, n_x)
t_test = torch.linspace(0, 1, n_t)
X_test, T_test = torch.meshgrid(x_test, t_test, indexing='ij')
xt_test = torch.stack([X_test.flatten(), T_test.flatten()], dim=1)

with torch.no_grad():
    u_pred = model_burgers(xt_test).reshape(n_x, n_t)

# Convert to numpy
X_np = X_test.numpy()
T_np = T_test.numpy()
u_pred_np = u_pred.numpy()

# Create visualization
fig = plt.figure(figsize=(16, 10))

# 3D surface plot
ax1 = fig.add_subplot(2, 3, 1, projection='3d')
surf = ax1.plot_surface(X_np, T_np, u_pred_np, cmap='viridis', alpha=0.8)
ax1.set_title('Burgers\' Equation Solution')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('u(x,t)')

# Contour plot
ax2 = fig.add_subplot(2, 3, 2)
contour = ax2.contourf(X_np, T_np, u_pred_np, levels=20, cmap='viridis')
ax2.set_title('Solution Evolution (Contour)')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
plt.colorbar(contour, ax=ax2)

# Solution at different times
ax3 = fig.add_subplot(2, 3, 3)
time_indices = [0, 10, 20, 30, 40, 49]
colors = plt.cm.viridis(np.linspace(0, 1, len(time_indices)))
for i, color in zip(time_indices, colors):
    ax3.plot(x_test.numpy(), u_pred_np[:, i], color=color, 
             label=f't = {t_test[i]:.2f}', linewidth=2)
ax3.set_xlabel('x')
ax3.set_ylabel('u(x,t)')
ax3.set_title('Solution at Different Times')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Initial condition comparison
ax4 = fig.add_subplot(2, 3, 4)
u_ic_pred = u_pred_np[:, 0]
u_ic_exact = -np.sin(np.pi * x_test.numpy())
ax4.plot(x_test.numpy(), u_ic_pred, 'b-', linewidth=2, label='PINN IC')
ax4.plot(x_test.numpy(), u_ic_exact, 'r--', linewidth=2, label='Exact IC')
ax4.set_xlabel('x')
ax4.set_ylabel('u(x,0)')
ax4.set_title('Initial Condition')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Training history
ax5 = fig.add_subplot(2, 3, 5)
ax5.semilogy(trainer_burgers.history['loss'], label='Total Loss')
ax5.semilogy(trainer_burgers.history['pde_loss'], label='PDE Loss')
ax5.semilogy(trainer_burgers.history['ic_loss'], label='IC Loss')
ax5.semilogy(trainer_burgers.history['bc_loss'], label='BC Loss')
ax5.set_xlabel('Epoch')
ax5.set_ylabel('Loss')
ax5.set_title('Training History')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Residual analysis
ax6 = fig.add_subplot(2, 3, 6)
# Compute PDE residual at test points
xt_test.requires_grad_(True)
u_test = model_burgers(xt_test)
u_x = compute_derivatives(u_test, xt_test, order=1)[0]
u_t = compute_derivatives(u_test, xt_test, order=1)[1]
u_xx = compute_derivatives(u_test, xt_test, order=2)[0]
residual = u_t + u_test * u_x - 0.01 * u_xx
residual_np = residual.detach().numpy().reshape(n_x, n_t)

im = ax6.imshow(np.abs(residual_np), aspect='auto', cmap='Reds', 
                extent=[-1, 1, 0, 1], origin='lower')
ax6.set_title('PDE Residual Magnitude')
ax6.set_xlabel('x')
ax6.set_ylabel('t')
plt.colorbar(im, ax=ax6)

plt.tight_layout()
plt.show()

print(f"Burgers' equation - Max residual: {np.abs(residual_np).max():.2e}")
print(f"Initial condition error: {np.mean((u_ic_pred - u_ic_exact)**2):.2e}")

## 7. Advanced Topics {#advanced-topics}

### 7.1 Adaptive Weights

One challenge in training PINNs is balancing different loss components. Here's an implementation of adaptive weight balancing:

In [ ]:
class AdaptiveWeightPINN:
    """
    PINN with adaptive loss weights based on the magnitude of gradients.
    """
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.weights = {'pde': 1.0, 'bc': 1.0, 'ic': 1.0}
        self.alpha = 0.9  # Smoothing factor for weight updates
    
    def update_weights(self, losses):
        """
        Update loss weights based on the rate of decrease of each loss component.
        """
        # Compute gradients for each loss component
        grad_norms = {}
        
        for key, loss in losses.items():
            if key != 'total':
                grad = torch.autograd.grad(loss, self.model.parameters(), 
                                         retain_graph=True, create_graph=False)
                grad_norm = torch.norm(torch.cat([g.flatten() for g in grad]))
                grad_norms[key] = grad_norm.item()
        
        # Update weights inversely proportional to gradient norms
        max_grad = max(grad_norms.values())
        for key in self.weights:
            if key in grad_norms:
                new_weight = max_grad / (grad_norms[key] + 1e-8)
                self.weights[key] = self.alpha * self.weights[key] + (1 - self.alpha) * new_weight
    
    def compute_loss(self, data):
        """
        Compute weighted loss with adaptive weights.
        """
        losses = {}
        
        # Compute individual losses (implementation depends on specific PDE)
        # This is a template - would be implemented for specific problems
        
        return losses

print("Adaptive weight balancing helps manage competing loss terms in complex PDEs.")

### 7.2 Fourier Neural Operators Integration

PINNs can be combined with Fourier Neural Operators for better performance on periodic problems:

In [ ]:
class FourierFeaturePINN(nn.Module):
    """
    PINN with Fourier feature encoding for better high-frequency representation.
    """
    def __init__(self, input_dim=1, hidden_dim=256, output_dim=1, 
                 fourier_dim=128, sigma=1.0):
        super(FourierFeaturePINN, self).__init__()
        
        # Random Fourier features
        self.B = nn.Parameter(torch.randn(input_dim, fourier_dim) * sigma)
        self.B.requires_grad = False
        
        # Network
        self.network = nn.Sequential(
            nn.Linear(2 * fourier_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def fourier_features(self, x):
        """Apply random Fourier features."""
        x_proj = torch.matmul(x, self.B) * 2 * np.pi
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)
    
    def forward(self, x):
        x_fourier = self.fourier_features(x)
        return self.network(x_fourier)

# Test Fourier feature PINN
fourier_model = FourierFeaturePINN(input_dim=1, fourier_dim=64)
test_input = torch.linspace(-1, 1, 100).reshape(-1, 1)
test_output = fourier_model(test_input)
print(f"Fourier PINN output shape: {test_output.shape}")
print("Fourier features help capture high-frequency components in solutions.")

### 7.3 Transfer Learning for PINNs

Pre-trained PINNs can be fine-tuned for similar problems:

In [ ]:
def transfer_learning_example():
    """
    Example of transfer learning with PINNs.
    """
    # Assume we have a pre-trained model for Poisson equation
    pretrained_model = SimplePINN(input_dim=1, hidden_dim=32, output_dim=1)
    
    # Fine-tune for a similar problem (e.g., Poisson with different boundary conditions)
    # Strategy 1: Freeze early layers, train only output layers
    for param in pretrained_model.network[:-2].parameters():
        param.requires_grad = False
    
    # Strategy 2: Lower learning rate for pretrained layers
    pretrained_params = list(pretrained_model.network[:-2].parameters())
    new_params = list(pretrained_model.network[-2:].parameters())
    
    optimizer = optim.Adam([
        {'params': pretrained_params, 'lr': 1e-5},
        {'params': new_params, 'lr': 1e-3}
    ])
    
    print("Transfer learning can accelerate training for similar PDE problems.")
    print("Strategies include freezing layers or using different learning rates.")

transfer_learning_example()

## 8. Conclusion {#conclusion}

In this tutorial, we've covered:

### Key Takeaways:
1. **PINNs embed physics directly**: PDEs become part of the loss function
2. **Automatic differentiation is crucial**: Enables computation of derivatives for PDE residuals
3. **Multiple loss components**: Balance PDE residuals, boundary conditions, and data fitting
4. **Architecture matters**: Different network designs for different problem types

### Best Practices:
- **Proper weight initialization**: Use Xavier or He initialization
- **Loss balancing**: Carefully balance different loss components
- **Sampling strategy**: Use appropriate collocation point sampling
- **Activation functions**: Tanh often works well for PINNs
- **Monitoring training**: Watch all loss components, not just total loss

### Applications:
- Fluid dynamics (Navier-Stokes equations)
- Heat transfer problems
- Wave propagation
- Inverse problems
- Multi-physics simulations

### Future Directions:
- **Improved architectures**: ResNets, attention mechanisms
- **Better optimization**: L-BFGS, adaptive learning rates
- **Uncertainty quantification**: Bayesian PINNs
- **Multi-scale problems**: Hierarchical approaches

PINNs represent a paradigm shift in computational physics, offering a flexible and powerful approach to solving PDEs. As the field continues to evolve, we can expect even more sophisticated techniques and broader applications.

### Next Steps:
1. Try implementing PINNs for your own PDE problems
2. Experiment with different architectures and loss formulations
3. Explore the other methods in this repository (DeepONet, FNO, Transformers)
4. Consider hybrid approaches combining multiple techniques

Happy learning and coding! 🚀